In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras_text_summarization.library.utility.plot_utils import plot_and_save_history
from keras_text_summarization.library.rnn import RecursiveRNN2
from keras_text_summarization.library.applications.fake_news_loader import fit_text
import numpy as np
import re
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
import pickle

In [ ]:
LOAD_EXISTING_WEIGHTS = False
np.random.seed(42)
data_dir_path = './data'
report_dir_path = './reports'
model_dir_path = './models/rnn2'

In [3]:
reviews = pd.read_csv(data_dir_path + "/Reviews.csv", encoding='utf-8')
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator',
                        'Time'], 1)
reviews = reviews.reset_index(drop=True)
# df.drop_duplicates(subset=["ctext"],inplace=True)
# df.dropna(inplace=True)
# df.reset_index(drop=True,inplace=True)

In [4]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [5]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'<br', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'>', ' ', text)
    text = re.sub(' +',' ',text)
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [8]:
clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [18]:
with open("amazon_texts_less", "wb") as fp:
    pickle.dump(cl_txt,fp)
with open("amazon_summaries_less", "wb") as fp2:
    pickle.dump(cl_summ,fp2)
with open("amazon_scores", "wb") as fp3:
    pickle.dump(cl_score,fp3)

In [ ]:
with open("amazon_texts", "rb") as fp:
    cl_txt = pickle.load(fp)
with open("amazon_summaries", "rb") as fp2:
    cl_summ = pickle.load(fp2)

In [9]:
len(clean_summaries)

568411

In [15]:
dumb=[]
dumb.append(reviews.Score[0])
dumb.append(reviews.Score[1])

In [16]:
dumb

[5, 1]

In [17]:
cl_summ =[]
cl_txt = []
cl_score=[]
for i in range(len(clean_summaries)):
    if(len(clean_texts[i].split())>10 and len(clean_texts[i].split())<30 and len(clean_summaries[i].split())<6 and len(clean_summaries[i].split())>4):
        if(any(char.isdigit() for char in clean_texts[i])==False):
            cl_txt.append(clean_texts[i])
            cl_summ.append(clean_summaries[i])
            cl_score.append(reviews.Score[i])

with open("amazon_texts_less", "wb") as fp:
    pickle.dump(cl_txt,fp)
with open("amazon_summaries_less", "wb") as fp2:
    pickle.dump(cl_summ,fp2)

t=Tokenizer()
t.fit_on_texts(cl_summ)
len(t.word_index)

print(cl_txt[500])
print(cl_summ[500])

max([len(txt.split()) for txt in cl_txt])

In [ ]:
Y = cl_summ
X = cl_txt
config = fit_text(X,Y)

In [ ]:
config['max_target_seq_length']

In [ ]:
summarizer = RecursiveRNN2(config)

In [ ]:
if LOAD_EXISTING_WEIGHTS:
    weight_file_path = RecursiveRNN2.get_weight_file_path(model_dir_path=model_dir_path)
    summarizer.load_weights(weight_file_path=weight_file_path)

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)
print('demo size: ', len(Xtrain))
print('testing size: ', len(Xtest))

In [ ]:
history = summarizer.fit(Xtrain, Ytrain, Xtest, Ytest, epochs=50, batch_size=128, model_dir_path=model_dir_path)

In [ ]:
summarizer.load_weights('./rnn2/recursive-rnn-2-28-2.5076-weights.h5')

In [ ]:
for i in np.random.permutation(np.arange(len(Xtrain)))[0:20]:
    x = X[i]
    actual_headline = Y[i]
    headline = summarizer.summarize(x)
    # print('Article: ', x)
    print('Original text: ', x)
    print('Generated Headline: ', headline)
    print('Original Headline: ', actual_headline)
    print('----------------------------------------')

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('RNN2_acc')
#plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('RNN2_loss')
#plt.show()

In [ ]:
history_plot_file_path = report_dir_path + '/' + RecursiveRNN2.model_name + '-history.png'
if LOAD_EXISTING_WEIGHTS:
    history_plot_file_path = report_dir_path + '/' + RecursiveRNN2.model_name + '-history-v' + str(summarizer.version) + '.png'
plot_and_save_history(history, summarizer.model_name, history_plot_file_path, metrics={'loss', 'acc'})